#code

In [6]:
import numpy as np
import pandas as pd
from dateutil import parser
from datetime import datetime

import openpyxl


def find_sheets_with_values(file_path, values_to_check):
    xls = pd.ExcelFile(file_path, engine='openpyxl')

    # List to hold names of sheets that contain any of the specified values
    sheets_with_values = []

    # Iterate through each sheet in the Excel file
    for sheet_name in xls.sheet_names:
        # Read the current sheet into a DataFrame
        df = pd.read_excel(xls, sheet_name=sheet_name)

        # Check if any of the specified values are in the DataFrame
        if df.isin(values_to_check).any().any():  # Checks all cells in the DataFrame
            sheets_with_values.append(sheet_name)

    return sheets_with_values


def load_sheet_to_df(file_path, sheet_name):
    # Load a specific worksheet into a DataFrame
    df = pd.read_excel(file_path, sheet_name=sheet_name, engine='openpyxl')
    return df



# Function to find the first occurrence of any keyword
def find_first_keyword(df, keywords):
    # Iterate over each row by its index
    for index, row in df.iterrows():
        # Check each cell in the row
        for col_index, (col_name, cell) in enumerate(row.items()):
            if cell in keywords:
                return index, col_name, col_index
    return None, None, None  # Return None if no keyword is found



# Function to extract all data from a specific starting point in the DataFrame
def extract_subset_df(df, start_row_index, start_col_index):
    # Slicing the DataFrame to get all rows and columns from the specified starting indices
    new_df = df.iloc[start_row_index:, start_col_index:]
    return new_df

def extract_first_nine_columns(df):
    # Slice to select the first 9 columns
    new_df = df.iloc[:, :9]  # "9" means up to but not including the 10th column
    return new_df

def remove_all_nan_rows(df):
    # Drop rows where all elements are NaN
    df_cleaned = df.dropna(how='all')
    return df_cleaned



# # Function to split the time range and format it, handling NaN values
# def split_and_format_time(df, column):
#     # Split the 'Time Range' into two separate columns
#     df[['From Time', 'To Time']] = df[column].str.split('-', expand=True)

#     # Format the 'From Time' and 'To Time' columns, adding handling for NaN values
#     df['SHIFT FROM'] = df['From Time'].apply(lambda x: f"{x[:2]}:{x[2:]}" if pd.notna(x) else np.nan)
#     df['SHIFT TO'] = df['To Time'].apply(lambda x: f"{x[:2]}:{x[2:]}" if pd.notna(x) else np.nan)

#     return df

# import pandas as pd
# import numpy as np







def convert_date_column(date_item):
    # Check if the item is already a datetime object
    if isinstance(date_item, datetime):
        return date_item.strftime('%Y-%m-%d')  # Format and return

    # If it's a string, check if it's already in 'YYYY-MM-DD' format
    try:
        if isinstance(date_item, str) and datetime.strptime(date_item, '%Y-%m-%d'):
            return date_item  # Return the date as is
    except ValueError:
        pass  # Not in 'YYYY-MM-DD' format, so proceed to parse and convert

    try:
        # Parse the date string into datetime
        dt = parser.parse(date_item)
        # Return formatted date
        return dt.strftime('%Y-%m-%d')
    except (ValueError, TypeError):
        # Return None or some default value if the date is invalid
        return None


In [7]:
import pandas as pd
from dateutil import parser



def split_and_format_time(df, column):
    # Initialize the 'data_check' column
    df['swift_check'] = np.where(df[column].isna(), 'Check', df[column])

    # Check if the value is properly formatted as '0000-0000'
    def check_format(value):
        if pd.isna(value):
            return 'Check'
        parts = value.split('-')
        if len(parts) != 2 or any(len(part) != 4 for part in parts):
            return 'Check'
        return value  # Return the original value if it's correctly formatted

    df['swift_check'] = df[column].apply(check_format)

    # Split the 'Time Range' into two separate columns
    df[['From Time', 'To Time']] = df[column].str.split('-', expand=True)

    # Format the 'From Time' and 'To Time' columns, adding handling for NaN values
    df['SHIFT FROM'] = df['From Time'].apply(lambda x: f"{x[:2]}:{x[2:]}" if pd.notna(x) else np.nan)
    df['SHIFT TO'] = df['To Time'].apply(lambda x: f"{x[:2]}:{x[2:]}" if pd.notna(x) else np.nan)

    return df



def format_and_check_dates(df, column_name):
      # Create a new column for the checks
      df['Data_check'] = 'Check'  # Default to 'Check'

      # Function to try and parse dates
      def parse_date(date):
          try:
              # Attempt to parse the date
              parsed_date = parser.parse(str(date), dayfirst=True)
              # Successfully parsed, format to 'YYYY-MM-DD'
              return parsed_date.strftime('%Y-%m-%d')
          except ValueError:
              # Parsing failed, return 'Check'
              return 'Check'

      # Apply the function to the specified column and store results in a new column
      df['Formatted_date'] = df[column_name].apply(parse_date)

      # Update 'Data_check' where dates are correctly parsed
      df['Data_check'] = df['Formatted_date'].apply(lambda x: 'OK' if x != 'Check' else 'Check')

      return df



def verify_product(df):
      # Calculate the product of HOURS and RATE
      df['Calculated_COST'] = df['HOURS_FINAL'] * df['RATE_FINAL']

      # Initialize the check column with 'OK' where the calculated product matches COST and handle NaNs
      df['HOURS_RATE_COST_check'] = np.where(
          (df['Calculated_COST'] == df['COST_FINAL']) &
          df['HOURS_FINAL'].notna() & df['RATE_FINAL'].notna() & df['COST_FINAL'].notna(),
          'OK',
          'Check'
      )

      # Optionally, drop the 'Calculated_COST' column if it's not needed
      df.drop(columns=['Calculated_COST'], inplace=True)

      return df


def clean_and_check_cost(df, column_name):
      # Clean the column by removing any non-numeric characters except the decimal point
      df['COST_Cleaned_Price'] = df[column_name].replace('[^\d.]', '', regex=True)

      # Convert the cleaned strings to numeric type (float)
      df['COST_FINAL'] = pd.to_numeric(df['COST_Cleaned_Price'], errors='coerce')

      # Create a new column to indicate the validity of the data
      df['COST_Data_check'] = 'OK'  # Default to 'OK'
      df.loc[df['COST_FINAL'].isna(), 'COST_Data_check'] = 'Check'  # Mark non-convertible or missing entries

      return df


def clean_and_check_rate(df, column_name):
      # Clean the column by removing any non-numeric characters except the decimal point
      df['RATE_Cleaned_Price'] = df[column_name].replace('[^\d.]', '', regex=True)

      # Convert the cleaned strings to numeric type (float)
      df['RATE_FINAL'] = pd.to_numeric(df['RATE_Cleaned_Price'], errors='coerce')

      # Create a new column to indicate the validity of the data
      df['RATE_Data_check'] = 'OK'  # Default to 'OK'
      df.loc[df['RATE_FINAL'].isna(), 'RATE_Data_check'] = 'Check'  # Mark non-convertible or missing entries

      return df

def clean_and_check_hours(df, column_name):
      # Clean the column by removing any non-numeric characters except the decimal point
      df['HOURS_Cleaned_Price'] = df[column_name].replace('[^\d.]', '', regex=True)

      # Convert the cleaned strings to numeric type (float)
      df['HOURS_FINAL'] = pd.to_numeric(df['HOURS_Cleaned_Price'], errors='coerce')

      # Create a new column to indicate the validity of the data
      df['HOURS_Data_check'] = 'OK'  # Default to 'OK'
      df.loc[df['HOURS_FINAL'].isna(), 'HOURS_Data_check'] = 'Check'  # Mark non-convertible or missing entries

      return df


def standardize_status_values(df, column_name):
      # Define the allowed values
      allowed_values = ['VACANT', 'NEW', 'PENDING','CURRENT']

      # Standardize the entries in the column
      df["STATUS_check"] = df[column_name].str.upper().str.strip()

      # Replace values not matching the allowed values with NaN or another placeholder
      df["STATUS_check"] = df[column_name].apply(lambda x: x if x in allowed_values else 'CHECK')

      return df



In [8]:
file_path="/content/data/HNELHD NON-SPECIALIST LOCUM VACANCY LIST (17) (1).xlsx"

In [9]:
values_to_check = ['VACANT', 'PENDING', 'NEW','CURRENT']
sheets_found = find_sheets_with_values(file_path, values_to_check)

##any form of  ['VACANT', 'PENDING', 'NEW','CURRENT'] check

FileNotFoundError: [Errno 2] No such file or directory: '/content/data/HNELHD NON-SPECIALIST LOCUM VACANCY LIST (17) (1).xlsx'

In [ ]:
sheets_found

In [ ]:
data_frame = load_sheet_to_df(file_path, "ARMIDALE CMO ED")
data_frame

In [ ]:
main_df=empty_df = pd.DataFrame(columns=["STATUS", "DATE", "SHIFT", "HOURS", "RATE", "COST", "ON_CALL", "ROLE","AREA"])


for i in sheets_found:
  try:
    data_frame = load_sheet_to_df(file_path, i)
    keywords = ['VACANT', 'PENDING', 'NEW','CURRENT']
    row_index, col_name, col_index = find_first_keyword(data_frame, keywords)
    row_index = row_index
    col_index = col_index
    new_df = extract_subset_df(data_frame, row_index, col_index)


    new_df_2 = new_df.dropna(axis=1, how='all')
    threshold = len(new_df_2.columns) / 2  # More than half the number of columns
    new_df_3 = new_df_2.dropna(thresh=threshold, axis=0)
    threshold = len(new_df_3) / 2  # More than half the number of rows
    new_df_4 = new_df_3.dropna(thresh=threshold, axis=1)

    new_column_names = ["STATUS", "DATE", "SHIFT", "HOURS", "RATE", "COST", "ON_CALL", "ROLE","AREA"]

    if len(new_column_names) == len(new_df_4.columns):
        new_df_4.columns = new_column_names
    else:
        print("Error: The number of new column names does not match the number of existing columns.")

    df_5=split_and_format_time(new_df_4,"SHIFT")
    df_6 = format_and_check_dates(df_5, 'DATE')
    df_7 = standardize_status_values(df_6, 'STATUS')
    df_8 = clean_and_check_hours(df_7, 'HOURS')
    df_9 = clean_and_check_rate(df_8, 'RATE')
    df_10 = clean_and_check_cost(df_9, 'COST')
    df_11 = verify_product(df_10)

    main_df = pd.concat([main_df, df_11], axis=0)
    print("sheet_done: ",i)

  except Exception as e:
    print(e,":",i)




In [ ]:
main_df

In [ ]:
{   }
{   }
{   }

In [ ]:


data_frame = load_sheet_to_df(file_path, "ARMIDALE O&G")
keywords = ['VACANT', 'PENDING', 'NEW','CURRENT']
row_index, col_name, col_index = find_first_keyword(data_frame, keywords)
row_index = row_index
col_index = col_index
new_df = extract_subset_df(data_frame, row_index, col_index)
new_df




In [ ]:
new_df_2 = new_df.dropna(axis=1, how='all')
threshold = len(new_df_2.columns) / 2  # More than half the number of columns
new_df_3 = new_df_2.dropna(thresh=threshold, axis=0)
threshold = len(new_df_3) / 2  # More than half the number of rows
new_df_4 = new_df_3.dropna(thresh=threshold, axis=1)
new_df_4

In [ ]:
new_column_names = ["STATUS", "DATE", "SHIFT", "HOURS", "RATE", "COST", "ON_CALL", "ROLE", "AREA"]

# Check if the length of new column names matches the number of columns in the DataFrame
if len(new_column_names) == len(new_df_4.columns):
    new_df_4.columns = new_column_names
else:
    print(f"Error: Expected {len(new_df_4.columns)} column names but got {len(new_column_names)}.")
    # Optionally, handle different cases:
    if len(new_column_names) < len(new_df_4.columns):
        # Add a placeholder for missing column names
        new_column_names += ['UNNAMED_' + str(i) for i in range(1, len(new_df_4.columns) - len(new_column_names) + 1)]
        new_df_4.columns = new_column_names
    else:
        # Truncate the list of new column names to match the DataFrame's column count
        new_df_4.columns = new_column_names[:len(new_df_4.columns)]


In [ ]:
new_df_4

In [ ]:
def split_and_format_time(df, column):
    # Initialize the 'data_check' column
    df['swift_check'] = np.where(df[column].isna(), 'Check', df[column])

    # Check if the value is properly formatted as '0000-0000'
    def check_format(value):
        if pd.isna(value):
            return 'Check'
        parts = value.split('-')
        if len(parts) != 2 or any(len(part) != 4 for part in parts):
            return 'Check'
        return value  # Return the original value if it's correctly formatted

    df['swift_check'] = df[column].apply(check_format)

    # Split the 'Time Range' into two separate columns
    df[['From Time', 'To Time']] = df[column].str.split('-', expand=True)

    # Format the 'From Time' and 'To Time' columns, adding handling for NaN values
    df['SHIFT FROM'] = df['From Time'].apply(lambda x: f"{x[:2]}:{x[2:]}" if pd.notna(x) else np.nan)
    df['SHIFT TO'] = df['To Time'].apply(lambda x: f"{x[:2]}:{x[2:]}" if pd.notna(x) else np.nan)

    return df



df_5=split_and_format_time(new_df_4,"SHIFT")
df_5

In [ ]:
import pandas as pd
from dateutil import parser

def format_and_check_dates(df, column_name):
    # Create a new column for the checks
    df['Data_check'] = 'Check'  # Default to 'Check'

    # Function to try and parse dates
    def parse_date(date):
        try:
            # Attempt to parse the date
            parsed_date = parser.parse(str(date), dayfirst=True)
            # Successfully parsed, format to 'YYYY-MM-DD'
            return parsed_date.strftime('%Y-%m-%d')
        except ValueError:
            # Parsing failed, return 'Check'
            return 'Check'

    # Apply the function to the specified column and store results in a new column
    df['Formatted_date'] = df[column_name].apply(parse_date)

    # Update 'Data_check' where dates are correctly parsed
    df['Data_check'] = df['Formatted_date'].apply(lambda x: 'OK' if x != 'Check' else 'Check')

    return df


# Apply the function
df_6 = format_and_check_dates(df_5, 'DATE')
df_6

In [ ]:
import pandas as pd

def standardize_status_values(df, column_name):
    # Define the allowed values
    allowed_values = ['VACANT', 'NEW', 'PENDING','CURRENT']

    # Standardize the entries in the column
    df["STATUS_check"] = df[column_name].str.upper().str.strip()

    # Replace values not matching the allowed values with NaN or another placeholder
    df["STATUS_check"] = df[column_name].apply(lambda x: x if x in allowed_values else 'CHECK')

    return df


# Apply the function
df_7 = standardize_status_values(df_6, 'STATUS')
df_7


In [ ]:
import pandas as pd

def clean_and_check_prices(df, column_name):
    # Clean the column by removing any non-numeric characters except the decimal point
    df['HOURS_Cleaned_Price'] = df[column_name].replace('[^\d.]', '', regex=True)

    # Convert the cleaned strings to numeric type (float)
    df['HOURS_FINAL'] = pd.to_numeric(df['HOURS_Cleaned_Price'], errors='coerce')

    # Create a new column to indicate the validity of the data
    df['HOURS_Data_check'] = 'OK'  # Default to 'OK'
    df.loc[df['HOURS_FINAL'].isna(), 'HOURS_Data_check'] = 'Check'  # Mark non-convertible or missing entries

    return df



# Apply the function
df_8 = clean_and_check_prices(df_7, 'HOURS')
df_8


In [ ]:
import pandas as pd

def clean_and_check_prices(df, column_name):
    # Clean the column by removing any non-numeric characters except the decimal point
    df['RATE_Cleaned_Price'] = df[column_name].replace('[^\d.]', '', regex=True)

    # Convert the cleaned strings to numeric type (float)
    df['RATE_FINAL'] = pd.to_numeric(df['RATE_Cleaned_Price'], errors='coerce')

    # Create a new column to indicate the validity of the data
    df['RATE_Data_check'] = 'OK'  # Default to 'OK'
    df.loc[df['RATE_FINAL'].isna(), 'RATE_Data_check'] = 'Check'  # Mark non-convertible or missing entries

    return df



# Apply the function
df_9 = clean_and_check_prices(df_8, 'RATE')
df_9

In [ ]:
import pandas as pd

def clean_and_check_prices(df, column_name):
    # Clean the column by removing any non-numeric characters except the decimal point
    df['COST_Cleaned_Price'] = df[column_name].replace('[^\d.]', '', regex=True)

    # Convert the cleaned strings to numeric type (float)
    df['COST_FINAL'] = pd.to_numeric(df['COST_Cleaned_Price'], errors='coerce')

    # Create a new column to indicate the validity of the data
    df['COST_Data_check'] = 'OK'  # Default to 'OK'
    df.loc[df['COST_FINAL'].isna(), 'COST_Data_check'] = 'Check'  # Mark non-convertible or missing entries

    return df



# Apply the function
df_10 = clean_and_check_prices(df_9, 'COST')
df_10

In [ ]:
import pandas as pd
import numpy as np

def verify_product(df):
    # Calculate the product of HOURS and RATE
    df['Calculated_COST'] = df['HOURS_FINAL'] * df['RATE_FINAL']

    # Initialize the check column with 'OK' where the calculated product matches COST and handle NaNs
    df['HOURS_RATE_COST_check'] = np.where(
        (df['Calculated_COST'] == df['COST_FINAL']) &
        df['HOURS_FINAL'].notna() & df['RATE_FINAL'].notna() & df['COST_FINAL'].notna(),
        'OK',
        'Check'
    )

    # Optionally, drop the 'Calculated_COST' column if it's not needed
    df.drop(columns=['Calculated_COST'], inplace=True)

    return df


# Apply the function
df_11 = verify_product(df_10)
df_11


In [ ]:
import os
import pandas as pd

folder_path = '/content/data'  # Change this to your specific folder path

# List all files in the directory
files = os.listdir(folder_path)

STATUS_list=[]
DATE_list=[]
SHIFT_list=[]
HOURS_list=[]
RATE_list=[]
COST_list=[]
ON_CALL_list=[]
ROLE_list=[]
AREA_list=[]

# Loop over the list of files
for file in files:
    if file.endswith('.xlsx'):
        file_path = os.path.join(folder_path, file)

        values_to_check = ['VACANT', 'PENDING', 'NEW','CURRENT']
        sheets_found = find_sheets_with_values(file_path, values_to_check)
        try:
          sheets_found.remove('Validation data')
        except:
          print("A")

        for i in sheets_found:
          print(i)
          print("---------------------")
          sheet_name = i
          data_frame = load_sheet_to_df(file_path, sheet_name)


          keywords = ['VACANT', 'PENDING', 'NEW','CURRENT']

          row_index, col_name, col_index = find_first_keyword(data_frame, keywords)

          row_index = row_index
          col_index = col_index


          new_df = extract_subset_df(data_frame, row_index, col_index)

          new_new_df = extract_first_nine_columns(new_df)
          final_df = remove_all_nan_rows(new_new_df)

          new_column_names = ["STATUS", "DATE", "SHIFT", "HOURS", "RATE", "COST", "ON_CALL", "ROLE","AREA"]

          if len(new_column_names) == len(final_df.columns):
              final_df.columns = new_column_names
          else:
              print("Error: The number of new column names does not match the number of existing columns.")

          # final_df['DATE'] = final_df['DATE'].apply(convert_date_column)

          # last_df = split_and_format_time(final_df, 'SHIFT')

          # new_df = final_df.drop(['SHIFT','From Time', 'To Time'], axis=1)

          for  k in new_column_names:
            try:
              unique_list = final_df[k].tolist()
              if k=="STATUS":
                  STATUS_list.extend(unique_list)

              if k=="DATE":
                  DATE_list.extend(unique_list)

              if k=="SHIFT":
                  SHIFT_list.extend(unique_list)

              if k=="HOURS":
                  HOURS_list.extend(unique_list)

              if k=="RATE":
                  RATE_list.extend(unique_list)

              if k=="COST":
                  COST_list.extend(unique_list)

              if k=="ON_CALL":
                  ON_CALL_list.extend(unique_list)

              if k=="ROLE":
                  ROLE_list.extend(unique_list)

              if k=="AREA":
                  AREA_list.extend(unique_list)

            except:
              print("error_at_sheet",sheet_name)
              print("file_name",file_path)






In [ ]:
STATUS_list=[]
DATE_list=[]
SHIFT_list=[]
HOURS_list=[]
RATE_list=[]
COST_list=[]
ON_CALL_list=[]
ROLE_list=[]
AREA_list=[]

In [ ]:
STATUS_list

In [ ]:
from collections import Counter

def count_unique_values(input_list):
    # Use Counter to count occurrences of each element
    count = Counter(input_list)

    # Print each unique value and its count
    for item, frequency in count.items():
        print(item,":" ,frequency)


my_list = [1, 2, 2, 3, 4, 4, 4, 5, 6, 6, 7]
count_unique_values(my_list)

In [ ]:

unique_list = list(dict.fromkeys(STATUS_list))
print(unique_list)
print(count_unique_values(STATUS_list))
# if not pending vacant new after making all capital show monitor.
#if empy need monitor

In [ ]:

unique_list = list(dict.fromkeys(DATE_list))
print(unique_list)
print(count_unique_values(DATE_list))

# make it in to data format if not valid and if it's empty need monitor

In [ ]:

unique_list = list(dict.fromkeys(SHIFT_list))
print(unique_list)
print(count_unique_values(SHIFT_list))

#if not in 0000-0000 format need monitor
#if any text inside need monitor

In [ ]:

unique_list = list(dict.fromkeys(HOURS_list))
print(unique_list)
print(count_unique_values(HOURS_list))

#not a number need monitor

In [ ]:

unique_list = list(dict.fromkeys(RATE_list))
print(unique_list)
print(count_unique_values(RATE_list))

#if 24 hours in it remove it and only take number
# If text there need monitor
#if nan monitor

In [ ]:

unique_list = list(dict.fromkeys(COST_list))
print(unique_list)
print(count_unique_values(COST_list))

#if 24 hours in it remove it and only take number
# If text there need monitor
#if nan monitor

In [ ]:

unique_list = list(dict.fromkeys(ON_CALL_list))
print(unique_list)
print(count_unique_values(ON_CALL_list))

#if anything otehr than YES or NO need monitor

In [ ]:

unique_list = list(dict.fromkeys(ROLE_list))
print(unique_list)
print(count_unique_values(ROLE_list))

# if have '1700-0859' like dara need monitor, also if we have Yes or NO dta need monitor
# if nan we need monitor

In [ ]:

unique_list = list(dict.fromkeys(AREA_list))
print(unique_list)
print(count_unique_values(AREA_list))

# if have '1700-0859' like dara need monitor, also if we have Yes or NO dta need monitor
# if nan we need monitor